In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import shap

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor

from sklearn.feature_selection import mutual_info_regression
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error


In [2]:
beer1=pd.read_csv("Beer/beers.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'Beer/beers.csv'

In [ ]:
beer2=pd.read_csv("Beer/beer_data_set.csv")

In [ ]:
beer2

,Name,key,Style,Style Key,Brewery,Description,ABV,Ave Rating,Min IBU,Max IBU,...,Body,Alcohol,Bitter,Sweet,Sour,Salty,Fruits,Hoppy,Spices,Malty
0,Amber,251,Altbier,8,Alaskan Brewing Co.,"Notes:Richly malty and long on the palate, wit...",5.3,3.65,25,50,...,32,9,47,74,33,0,33,57,8,111
1,Double Bag,252,Altbier,8,Long Trail Brewing Co.,"Notes:This malty, full-bodied double alt is al...",7.2,3.90,25,50,...,57,18,33,55,16,0,24,35,12,84
2,Long Trail Ale,253,Altbier,8,Long Trail Brewing Co.,Notes:Long Trail Ale is a full-bodied amber al...,5.0,3.58,25,50,...,37,6,42,43,11,0,10,54,4,62
3,Doppelsticke,254,Altbier,8,Uerige Obergärige Hausbrauerei,Notes:,8.5,4.15,25,50,...,55,31,47,101,18,1,49,40,16,119
4,Scurry,255,Altbier,8,Off Color Brewing,Notes:Just cause it's dark and German doesn't ...,5.3,3.67,25,50,...,69,10,63,120,14,0,19,36,15,218
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5553,Red Nose Winter Ale,696,Winter Warmer,17,Natty Greene's Pub & Brewing Co.,Notes:,6.8,3.59,35,50,...,44,24,19,52,21,0,26,21,96,77
5554,Our Special Ale 2019 (Anchor Christmas Ale),697,Winter Warmer,17,Anchor Brewing Company,Notes:This is the forty-fifth annual Our Speci...,6.9,3.79,35,50,...,73,9,58,87,23,0,48,91,44,104
5555,Fish Tale Winterfish,698,Winter Warmer,17,Fish Brewing Company / Fishbowl Brewpub,Notes:,7.5,3.76,35,50,...,36,50,70,72,59,0,81,110,18,73
5556,"He'Brew Hanukkah, Chanukah: Pass The Beer",699,Winter Warmer,17,Shmaltz Brewing Company,Notes:Chanukah Beer pours a rich crystal clear...,8.0,3.61,35,50,...,64,30,57,78,15,1,28,57,23,129


In [ ]:
beer1.columns=[col.replace(" ","_").lower() for col in beer1.columns]
beer2.columns=[col.replace(" ","_").lower() for col in beer2.columns]
beer2["ibu"] = beer2[["min_ibu", "max_ibu"]].mean(axis=1)
beer=pd.merge(beer2,beer1,on="name", how="left")

In [ ]:
beer
beer.drop(columns=["key","brewery","description","min_ibu","max_ibu","unnamed:_0"], inplace=True)

In [ ]:
beer.drop(columns=["ibu_x","abv_y","ibu_y","id","style_y","brewery_id","ounces"], inplace=True)

In [ ]:
beer.nunique()

name           5317
style_x         112
style_key       112
abv_x           229
ave_rating      289
astringency      73
body            165
alcohol         116
bitter          141
sweet           191
sour            222
salty            47
fruits          185
hoppy           156
spices          155
malty           217
dtype: int64

In [ ]:
beer.isna().sum()

name           2
style_x        0
style_key      0
abv_x          0
ave_rating     0
astringency    0
body           0
alcohol        0
bitter         0
sweet          0
sour           0
salty          0
fruits         0
hoppy          0
spices         0
malty          0
dtype: int64

In [ ]:
beer.shape

(5663, 16)

In [ ]:
beer

,name,style_x,style_key,abv_x,ave_rating,astringency,body,alcohol,bitter,sweet,sour,salty,fruits,hoppy,spices,malty
0,Amber,Altbier,8,5.3,3.65,13,32,9,47,74,33,0,33,57,8,111
1,Double Bag,Altbier,8,7.2,3.90,12,57,18,33,55,16,0,24,35,12,84
2,Long Trail Ale,Altbier,8,5.0,3.58,14,37,6,42,43,11,0,10,54,4,62
3,Doppelsticke,Altbier,8,8.5,4.15,13,55,31,47,101,18,1,49,40,16,119
4,Scurry,Altbier,8,5.3,3.67,21,69,10,63,120,14,0,19,36,15,218
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5658,Red Nose Winter Ale,Winter Warmer,17,6.8,3.59,8,44,24,19,52,21,0,26,21,96,77
5659,Our Special Ale 2019 (Anchor Christmas Ale),Winter Warmer,17,6.9,3.79,22,73,9,58,87,23,0,48,91,44,104
5660,Fish Tale Winterfish,Winter Warmer,17,7.5,3.76,11,36,50,70,72,59,0,81,110,18,73
5661,"He'Brew Hanukkah, Chanukah: Pass The Beer",Winter Warmer,17,8.0,3.61,6,64,30,57,78,15,1,28,57,23,129


In [ ]:
beer.rename(columns= {"style_x": "type","abv_x":"abv","ave_rating":"rating"}, inplace=True)

In [ ]:
beertype = pd.DataFrame({
    "type": beer["type"],
    "style_key": beer["style_key"]
})

beertype

,type,style_key
0,Altbier,8
1,Altbier,8
2,Altbier,8
3,Altbier,8
4,Altbier,8
...,...,...
5658,Winter Warmer,17
5659,Winter Warmer,17
5660,Winter Warmer,17
5661,Winter Warmer,17


In [ ]:

with open('Beer/recipeData.csv', 'r', encoding='utf-8', errors='replace') as file:
    content = file.read()

In [ ]:
beer6=pd.read_csv("Beer/Product_range.csv")

In [ ]:
beer6

,Product_code,Vendor_code,Name,Retail_price,Base_unit,Country_of_Origin,Size,ABV
0,5028,NaN,1 Symbiotica Apple 0.375,300.0,Pieces,Russia,0.375,4.5
1,4846,NaN,1 Symbiotica Hard Kombucha Renegade Aronia 0.33,200.0,Pieces,Russia,0.330,3.0
2,1340,NaN,1000 IBU Imperial IPA Barrel l,960.0,Liters,NaN,1.000,9.6
3,4372,NaN,18th Street Brewery,NaN,NaN,USA,NaN,NaN
4,4302,NaN,18th Street Brewery Deal With The Devil 0.473,630.0,Pieces,USA,0.473,8.5
...,...,...,...,...,...,...,...,...
5309,868,NaN,Spagetti Vestern,880.0,Liters,NaN,1.000,8.7
5310,1861,NaN,Stoun Imperial Rashn Stout Barrel l,1200.0,Liters,NaN,1.000,10.8
5311,4724,NaN,Semjuel Adams Boston Lager Barrel,720.0,Liters,NaN,1.000,4.8
5312,822,Bakunin,Bakunin Urban Juice,600.0,Liters,Russia,1.000,7.2


In [ ]:
beer.duplicated().sum()
beer.drop_duplicates(inplace=True)

In [ ]:
beer.shape
beer.duplicated().sum()

0

In [ ]:
beer.rename(columns= {"style_key": "type_id"}, inplace=True)

In [ ]:
beer_rat=pd.read_csv("Beer/beer_profile_and_ratings.csv")

In [ ]:
beer_rat.columns=[col.replace(" ","_").lower() for col in beer_rat.columns]
beer_rat["ibu"] = beer_rat[["min_ibu", "max_ibu"]].mean(axis=1)

In [ ]:
beer_rat.drop(columns=["beer_name_(full)","description","min_ibu","max_ibu"], inplace=True)
beer_rat.rename(columns= {"style": "type","review_overall":"rating"}, inplace=True)

In [ ]:
beer_rat.drop(columns=["brewery"], inplace=True)

In [ ]:
beer.isna().sum()

name           2
type           0
type_id        0
abv            0
rating         0
astringency    0
body           0
alcohol        0
bitter         0
sweet          0
sour           0
salty          0
fruits         0
hoppy          0
spices         0
malty          0
dtype: int64

In [ ]:
beer.dropna(inplace=True)

In [ ]:
beer.shape

(5556, 16)

In [ ]:
features = beer.drop(columns=["type_id","name","type"]) 
target = beer["rating"]
x_train,x_test,y_train,y_test=train_test_split(features, target, test_size=0.20,random_state=0)
beer

,name,type,type_id,abv,rating,astringency,body,alcohol,bitter,sweet,sour,salty,fruits,hoppy,spices,malty
0,Amber,Altbier,8,5.3,3.65,13,32,9,47,74,33,0,33,57,8,111
1,Double Bag,Altbier,8,7.2,3.90,12,57,18,33,55,16,0,24,35,12,84
2,Long Trail Ale,Altbier,8,5.0,3.58,14,37,6,42,43,11,0,10,54,4,62
3,Doppelsticke,Altbier,8,8.5,4.15,13,55,31,47,101,18,1,49,40,16,119
4,Scurry,Altbier,8,5.3,3.67,21,69,10,63,120,14,0,19,36,15,218
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5658,Red Nose Winter Ale,Winter Warmer,17,6.8,3.59,8,44,24,19,52,21,0,26,21,96,77
5659,Our Special Ale 2019 (Anchor Christmas Ale),Winter Warmer,17,6.9,3.79,22,73,9,58,87,23,0,48,91,44,104
5660,Fish Tale Winterfish,Winter Warmer,17,7.5,3.76,11,36,50,70,72,59,0,81,110,18,73
5661,"He'Brew Hanukkah, Chanukah: Pass The Beer",Winter Warmer,17,8.0,3.61,6,64,30,57,78,15,1,28,57,23,129


In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error


standardizer=StandardScaler()
standardizer.fit(x_train)
x_train_st = standardizer.transform(x_train)
x_test_st= standardizer.transform(x_test)
pd.DataFrame(x_train_st, columns=x_train.columns)

,abv,rating,astringency,body,alcohol,bitter,sweet,sour,salty,fruits,hoppy,spices,malty
0,0.142733,0.294061,0.430338,-0.333712,-0.110620,0.581270,0.328011,0.406994,-0.338250,0.665193,1.001920,0.593491,-0.954857
1,0.536914,0.721408,-0.255887,0.445797,0.488954,-0.903756,0.551925,-0.297654,-0.081822,0.801835,-0.740912,0.676898,0.474029
2,1.719455,1.283707,-0.684777,0.268636,3.159788,-0.346871,1.391603,0.004338,-0.338250,0.774507,-0.647546,0.635194,0.474029
3,0.063897,0.451504,-0.513221,0.162339,-0.328648,0.544145,1.279646,-0.272488,-0.338250,0.063967,0.503968,0.051341,1.018367
4,0.142733,0.226585,0.859229,-0.617170,0.815995,-1.089384,-0.595635,2.067952,-0.081822,-0.236646,-0.865401,0.593491,-0.909496
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4439,1.325274,0.721408,-0.084330,0.516661,0.870502,-0.681002,2.315249,0.054670,-0.338250,1.184433,-0.772034,0.635194,0.451349
4440,-0.645627,-0.583126,0.430338,0.197771,0.270927,0.804024,0.356000,-0.121492,-0.338250,-0.154660,1.219774,-0.198881,0.768879
4441,0.103315,-0.245746,0.344560,-0.333712,-0.328648,1.435160,-0.707592,0.004338,0.174607,-0.100003,1.655482,-0.407400,-0.342477
4442,-0.448537,-0.245746,0.001448,-0.156551,-0.383154,0.098637,0.216054,-0.524149,-0.081822,-0.509930,0.503968,-0.449104,0.927644


In [ ]:
def score_model(x_train_st, y_train, model):
    log_y = np.log(y_train)
    scores = cross_val_score(
        model, x_train_st, log_y, cv=8, scoring='neg_mean_squared_error'
    )
    scores = np.mean(abs(scores))
    scores = np.sqrt(scores)
    return scores

In [ ]:
models = [LinearRegression(), DecisionTreeRegressor(), RandomForestRegressor(),AdaBoostRegressor(), GradientBoostingRegressor(), XGBRegressor()]

model_scores = []
for model in models:
    model_score = score_model(x_train_st, y_train, model)
    model_scores.append(model_score)
    print(model,'RMSE score:', model_score)
    print('\n')
    
print("Best score (RMSE):", min(model_scores))
print("Best model:", models[model_scores.index(min(model_scores))])


LinearRegression() RMSE score: 0.018211764269732984


DecisionTreeRegressor() RMSE score: 0.003242976267008452


RandomForestRegressor() RMSE score: 0.0030518790160205473


AdaBoostRegressor() RMSE score: 0.014322695748374178


GradientBoostingRegressor() RMSE score: 0.0028514141304882175


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_paral

In [ ]:
GB= GradientBoostingRegressor(n_estimators=100,
                              max_depth=20)


In [ ]:
GB.fit(x_train_st, y_train)

GradientBoostingRegressor(max_depth=20)

In [ ]:
pred=GB.predict(x_test_st)
GB.score(x_train_st,y_train)


0.9999999992944921

In [ ]:
np.sqrt(mean_squared_error(y_test,pred))

0.002380025252015995

In [ ]:
beer_pred=pd.DataFrame(x_test_st)
beer_pred["pred"]=pred
beer_pred["real"]=y_train
beer_pred

,0,1,2,3,4,5,6,7,8,9,10,11,12,pred,real
0,-0.758477,-0.610929,-0.347267,0.697129,-0.488553,-0.498265,-0.335802,-0.597612,-0.081054,-0.477487,-0.896789,-0.528208,0.025465,3.490007,3.65
1,-0.246886,-0.249829,-0.347267,0.200025,-0.543237,0.464060,-0.195536,-0.194348,-0.342212,-0.504855,1.145356,-0.320074,-0.382832,3.650003,3.90
2,-1.230715,0.246683,0.594281,0.626115,-0.379186,1.796511,0.505795,0.536569,-0.342212,0.507761,2.413961,-0.070313,1.023527,3.869997,3.58
3,-0.443652,-0.994597,0.508685,-0.332587,0.003600,-0.794366,3.002535,0.687793,-0.342212,2.532993,-0.680198,-0.278447,-0.133317,3.320012,4.15
4,-0.443652,-0.091848,0.423090,0.981189,-0.051084,0.760161,-0.027216,0.536569,0.702421,0.863545,1.238181,-0.486581,-0.269416,3.720001,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1662,2.350423,-0.430379,-1.288815,-0.332587,1.206640,-0.942416,0.421636,-0.269960,-0.342212,-0.340647,-1.020555,-0.694716,-0.723081,3.570005,NaN
1663,0.973063,0.607783,-0.261672,1.087712,0.222334,0.316010,0.253316,1.519526,0.180105,0.015137,-0.680198,-0.320074,0.615229,4.029993,3.99
1664,0.067940,0.043565,-0.090481,2.792070,-0.215135,0.834186,0.561902,-0.723632,-0.342212,-0.888007,-0.339840,1.053612,1.227676,3.779999,3.70
1665,0.067940,0.923746,0.251900,-0.155050,-0.051084,1.167298,-0.700495,-0.269960,-0.342212,-0.395383,1.640421,-0.528208,-0.473565,4.169989,4.05


In [ ]:
beer

,name,type,type_id,abv,rating,astringency,body,alcohol,bitter,sweet,sour,salty,fruits,hoppy,spices,malty
0,Amber,Altbier,8,5.3,3.65,13,32,9,47,74,33,0,33,57,8,111
1,Double Bag,Altbier,8,7.2,3.90,12,57,18,33,55,16,0,24,35,12,84
2,Long Trail Ale,Altbier,8,5.0,3.58,14,37,6,42,43,11,0,10,54,4,62
3,Doppelsticke,Altbier,8,8.5,4.15,13,55,31,47,101,18,1,49,40,16,119
4,Scurry,Altbier,8,5.3,3.67,21,69,10,63,120,14,0,19,36,15,218
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5658,Red Nose Winter Ale,Winter Warmer,17,6.8,3.59,8,44,24,19,52,21,0,26,21,96,77
5659,Our Special Ale 2019 (Anchor Christmas Ale),Winter Warmer,17,6.9,3.79,22,73,9,58,87,23,0,48,91,44,104
5660,Fish Tale Winterfish,Winter Warmer,17,7.5,3.76,11,36,50,70,72,59,0,81,110,18,73
5661,"He'Brew Hanukkah, Chanukah: Pass The Beer",Winter Warmer,17,8.0,3.61,6,64,30,57,78,15,1,28,57,23,129


ImportError: cannot import name 'shap_values' from 'shap' (c:\Users\valte\Documents\Bootcamp\Lib\site-packages\shap\__init__.py)

In [ ]:
beer.to_csv("beer.csv")

In [ ]:
beer_gross=pd.read_csv("Beer/beer_profile_and_ratings.csv")

In [ ]:
beer_gross.to_csv("beer_gross.csv")